In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import vosk
import pyttsx3

In [2]:
df = pd.read_csv('Conversations - Sheet1.csv')

In [3]:
df.head()

,Sentences,Intent,Responses
0,Good Morning,Greeting,Good morning. How are you today?
1,I am fine,Greeting,"Oh. great, That is good to hear. How can i hel..."
2,Thank you,Greeting,"You are welcome, is there anything you would l..."
3,How are you?,Greeting,I am fine. Thank you and you?
4,Good bye,goodbye,goodbye


In [4]:
def get_response(text):
    user_speech = [text]
    sentences = list(df['Sentences']) + user_speech
    bag_of_words = TfidfVectorizer().fit_transform(sentences)
#     print(sentences)
    similarity_scores = cosine_similarity(bag_of_words)
    similarity_scores = similarity_scores[-1][:-1]
    idx = np.argmax(similarity_scores)
    score = np.max(similarity_scores)
    if score >=0.6:
        row = df.iloc[idx]
        response = row['Responses']
        intent = row['Intent']
    else:
        response = """I'm sorry, i don't understand. Can you rephrase your statement, i'm just a bot undergoing training"""
        intent = None
    return (response,intent)
    

In [5]:
get_response('Good evening')

('Good Evening. How are you today?', 'Greeting')

In [6]:
def speak(text):
    engine = pyttsx3.init()
    rate = engine.getProperty('rate')
    engine.setProperty('rate', rate-30)
    engine.say(text)
    engine.runAndWait()

In [7]:
speak(get_response(""))

In [8]:
import argparse
import os
import queue
import sounddevice as sd
import vosk
import sys
import ast
import json

q = queue.Queue()

def int_or_str(text):
    """Helper function for argument parsing."""
    try:
        return int(text)
    except ValueError:
        return text

def callback(indata, frames, time, status):
    """This is called (from a separate thread) for each audio block."""
    if status:
        print(status, file=sys.stderr)
    q.put(bytes(indata))

parser = argparse.ArgumentParser(add_help=False)
parser.add_argument(
    '-l', '--list-devices', action='store_true',
    help='show list of audio devices and exit')
args, remaining = parser.parse_known_args()
if args.list_devices:
    print(sd.query_devices())
    parser.exit(0)
parser = argparse.ArgumentParser(
    description=__doc__,
    formatter_class=argparse.RawDescriptionHelpFormatter,
    parents=[parser])
parser.add_argument(
    '-f', '--filename', type=str, metavar='FILENAME',
    help='audio file to store recording to')
parser.add_argument(
    '-m', '--model', type=str, metavar='MODEL_PATH',
    help='Path to the model')
parser.add_argument(
    '-d', '--device', type=int_or_str,
    help='input device (numeric ID or substring)')
parser.add_argument(
    '-r', '--samplerate', type=int, help='sampling rate')
args = parser.parse_args(remaining)

try:
    if args.model is None:
        args.model = "model"
    if not os.path.exists(args.model):
        print ("Please download a model for your language from https://alphacephei.com/vosk/models")
        print ("and unpack as 'model' in the current folder.")
        parser.exit(0)
    if args.samplerate is None:
        device_info = sd.query_devices(args.device, 'input')
        # soundfile expects an int, sounddevice provides a float:
        args.samplerate = int(device_info['default_samplerate'])

    model = vosk.Model(args.model)

    if args.filename:
        dump_fn = open(args.filename, "wb")
    else:
        dump_fn = None

    with sd.RawInputStream(samplerate=args.samplerate, blocksize = 8000, device=args.device, dtype='int16',
                            channels=1, callback=callback):
            print('#' * 80)
            print('Press Ctrl+C to stop the recording')
            print('#' * 80)

            rec = vosk.KaldiRecognizer(model, args.samplerate)
            while True:
                data = q.get()
                if rec.AcceptWaveform(data):
                    speech =json.loads (rec.Result())['text']
                    #result = json.loads(result)['text']
                    print(f' Me: {speech}')
#                     print(type(result))
                    reply, intention = get_response(speech)
                    print(f'Bot: {reply}')
                    speak(reply)
                    with q.mutex:
                        q.queue.clear()
                    if intention != None:
                        if intention.lower()=="goodbye" or intention.lower()=="good bye":
                            break
                          
#                 else:
#                     print(rec.PartialResult().split()[-1])
#                 if dump_fn is not None:
#                     dump_fn.write(data)

except KeyboardInterrupt:
    print('\nDone')
    parser.exit(0)
except Exception as e:
    parser.exit(type(e).__name__ + ': ' + str(e))

################################################################################
Press Ctrl+C to stop the recording
################################################################################
 Me: pay good money
Bot: I'm sorry, i don't understand. Can you rephrase your statement, i'm just a bot undergoing training
 Me: good morning
Bot: Good morning. How are you today?
 Me: i am fine and you
Bot: Oh. great, That is good to hear. How can i help you?
 Me: 
Bot: I'm sorry, i don't understand. Can you rephrase your statement, i'm just a bot undergoing training
 Me: 
Bot: I'm sorry, i don't understand. Can you rephrase your statement, i'm just a bot undergoing training
 Me: the good bye
Bot: goodbye
